In [2]:
import numpy as np
import cython
import time
%load_ext Cython
import im2col_cython
import cython_weave

In [4]:
def array_weave_forwards(arr, weave_param):
    """
    input array size = (num_img, num_filters, height, width)
    output array size = (num_img, num_filters, 
                        height*(num_zeros + 1) - num_zeros, 
                        width*(num_zeros + 1) - num_zeros)
    Ex: 
    a = [[ 1  2  3  4  5]
        [ 6  7  8  9 10]
        [11 12 13 14 15]
        [16 17 18 19 20]
        [21 22 23 24 25]]
    >>> b = zero_weave(a, {'num_zeros':2, 'filter_size':3})
    b = [[ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
        [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
        [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [11. 14.  0. 12. 15.  0. 13.  0. 11. 14.  0. 12. 15.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 1.  4.  0.  2.  5.  0.  3.  0.  1.  4.  0.  2.  5.]
        [16. 19.  0. 17. 20.  0. 18.  0. 16. 19.  0. 17. 20.]
        [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
        [ 6.  9.  0.  7. 10.  0.  8.  0.  6.  9.  0.  7. 10.]
        [21. 24.  0. 22. 25.  0. 23.  0. 21. 24.  0. 22. 25.]]

    
    """
    num_zeros = weave_param['num_zeros']
    filter_size = weave_param['filter_size']
    cache = (arr, weave_param)
    
    num_img, num_filters, height, width = arr.shape
    
    out = np.zeros([num_img,
                      num_filters,
                      height*(num_zeros + 1) - num_zeros,
                      width*(num_zeros + 1) - num_zeros])
    #This needs to be generalized to other filter sizes
    expand_dist = 2*filter_size+2
    slice_jump = num_zeros + 1
    
    for i in range(height):
        for j in range(width):
            temp_val = arr[:,:,i,j]
            big_i = filter_size * i 
            big_j = filter_size * j
            for i_change in [-expand_dist, 0, expand_dist]:
                for j_change in [-expand_dist, 0, expand_dist]:
                    if (min(big_i+i_change,big_j+j_change) >= 0 
                        and max(big_i+i_change,big_j+j_change) < height*(num_zeros + 1) - num_zeros):
                        out[:,:,big_i+i_change,big_j+j_change] = temp_val

    out[:,:,::slice_jump,::slice_jump] = 0
                     
    return out, cache

def array_weave_backwards(dx, cache):
    org_arr, weave_param = cache
    num_zeros = weave_param['num_zeros']
    filter_size = weave_param['filter_size']
    
    expand_dist = 2*filter_size+2
    slice_jump = num_zeros + 1
    
    num_img, num_filters, height, width = org_arr.shape
    
    dout = np.zeros([num_img,
                   num_filters,
                   height,
                   width]) 
    for i in range(height):
        for j in range(width):
            big_i = filter_size * i 
            big_j = filter_size * j
            for i_change in [-expand_dist, 0, expand_dist]:
                for j_change in [-expand_dist, 0, expand_dist]:
                    if (min(big_i+i_change,big_j+j_change) >= 0 
                        and max(big_i+i_change,big_j+j_change) < height*(num_zeros + 1) - num_zeros):
                        if i_change != 0 and j_change != 0:
                            dout[:,:,i,j] += dx[:,:,big_i+i_change,big_j+j_change]
    
    return dout

In [41]:
%%cython -a
cimport numpy as np
cimport cython
import numpy as np

# DTYPE = np.float64
# ctypedef np.float64_t DTYPE_t

ctypedef fused DTYPE_t:
    np.float32_t
    np.float64_t
    
@cython.boundscheck(False)
def cython_array_weave_backwards(np.ndarray[DTYPE_t, ndim=4] dx,
                                  int num_img, int num_filters,
                                  int height, int width,
                                  int num_zeros, int filter_size):
    
    cdef int expand_dist = 2*filter_size + 2
    cdef int slice_jump = num_zeros + 1
    cdef int HH = dx.shape[2]

    cdef int img, layer, i, j, big_i, big_j
    cdef int i_change, j_change
    
    cdef np.float64_t temp_val
    cdef int temp_x
    cdef int temp_y

    cdef np.ndarray[DTYPE_t, ndim=4] dout = np.zeros([num_img,
                                                num_filters,
                                                height,
                                                width])
    for img in range(num_img):
        for layer in range(num_filters):
            for i in range(height):
                for j in range(width):
                    big_i = filter_size * i 
                    big_j = filter_size * j
                    i_change, j_change = -expand_dist, -expand_dist
                    while i_change < expand_dist:
                        while j_change < expand_dist:
                            if (min(big_i+i_change,big_j+j_change) >= 0 
                                and max(big_i+i_change,big_j+j_change) < HH):
                                    if i_change != 0 and j_change != 0:
                                        dout[img,layer,i,j] += dx[img,layer,big_i+i_change,big_j+j_change]
                            j_change += expand_dist
                        i_change += expand_dist
    return dout


In [4]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

# DTYPE = np.float64
# ctypedef np.float64_t DTYPE_t

ctypedef fused DTYPE_t:
    np.float32_t
    np.float64_t
    
@cython.boundscheck(False) 
@cython.wraparound(False)
@cython.cdivision(True)
def cython_2d_zero_weave_forward(np.ndarray[DTYPE_t, ndim=2] arr,
                                int num_zeros, int filter_size):
    """
    Cython funciton for preforming array_weave forwards that is much faster!
    """
    cdef int height = arr.shape[0]
    cdef int width = arr.shape[1] 
    
    cdef int slice_jump = num_zeros + 1

    cdef int i, j, big_i, big_j
    
    cdef int HH = height*(num_zeros + 1) - num_zeros
    cdef int WW = width*(num_zeros + 1) - num_zeros

    cdef np.ndarray[DTYPE_t, ndim=2] out = np.zeros([HH, WW])


    for i in range(height):
        for j in range(width):
            big_i = slice_jump * i 
            big_j = slice_jump * j
            out[big_i, big_j] = arr[i,j]
    return out

In [53]:
%%cython -a
import numpy as np
cimport numpy as np
cimport cython

# DTYPE = np.float64
# ctypedef np.float64_t DTYPE_t

ctypedef fused DTYPE_t:
    np.float32_t
    np.float64_t
    
@cython.boundscheck(False) 
@cython.wraparound(False)
@cython.cdivision(True)
def cython_array_weave_forward(np.ndarray[DTYPE_t, ndim=4] arr,
                                np.ndarray[DTYPE_t, ndim=4] out,
                                int num_zeros, int filter_size):
    """
    Cython funciton for preforming array_weave forwards that is much faster!
    """
    cdef int num_img = arr.shape[0]
    cdef int num_filters = arr.shape[1]
    cdef int height = arr.shape[2]
    cdef int width = arr.shape[3] 
    
    cdef int expand_dist = 2*filter_size + 2
    cdef int slice_jump = num_zeros + 1

    cdef int img, layer, i, j, big_i, big_j, i_change, j_change
    
    cdef np.float64_t temp_val
    
    cdef int HH = height*(num_zeros + 1) - num_zeros
    cdef int WW = width*(num_zeros + 1) - num_zeros
    
    cdef int temp_x
    cdef int temp_y

    for img in range(num_img):
        for layer in range(num_filters):
            for i in range(height):
                for j in range(width):
                    temp_val = arr[img,layer,i,j]
                    big_i = filter_size * i 
                    big_j = filter_size * j
                    #######
#                     i_change, j_change = -expand_dist, -expand_dist
#                     while i_change < expand_dist:
#                         while j_change < expand_dist:
#                             if (min(big_i+i_change,big_j+j_change) >= 0 
#                                 and max(big_i+i_change,big_j+j_change) < HH):
#                                     if i_change != 0 and j_change != 0:
#                                         out[img,layer,big_i+i_change,big_j+j_change] = temp_val
#                             j_change += expand_dist
#                         i_change += expand_dist
                    #First Column
                    temp_x = big_i-expand_dist
                    temp_y = big_j-expand_dist
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val
                    temp_x = big_i
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val
                    temp_x = big_i + expand_dist 
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        pass
                        out[img,layer,temp_x,temp_y] = temp_val
                    #Second Column (Only 2 Points)
                    temp_x = big_i-expand_dist
                    temp_y = big_j
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val
                    temp_x = big_i + expand_dist 
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val
                    #Third Column
                    temp_x = big_i-expand_dist
                    temp_y = big_j+expand_dist
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val
                    temp_x = big_i
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val
                    temp_x = big_i + expand_dist 
                    if (min(temp_x,temp_y) >= 0 and max(temp_x,temp_y) < HH):
                        out[img,layer,temp_x,temp_y] = temp_val

    return out



In [20]:
num_img = 10
num_filters = 50
height = width = 32
X = np.random.randn(num_img,num_filters,width,width)
weave_param = {'num_zeros':2, 'filter_size':3}
HH = height*(2 + 1) - 2
out = np.zeros([num_img,num_filters,HH,HH])

def array_weave_fast_forward(X, weave_param):
    out = np.zeros([num_img,num_filters,HH,HH])
    cache = (X, weave_param)
    out =  cython_array_weave_forward(X,out, weave_param['num_zeros'],
                                      weave_param['filter_size'])
    return out, cache

def array_weave_fast_backwards(dx, cache):
    X, weave_param = cache
    num_img, num_filters, height, width = X.shape
    num_zeros = weave_param['num_zeros']
    filter_size = weave_param['filter_size']
    dout = cython_array_weave_backwards(dx, num_img, num_filters, 
                                        height, width, num_zeros, filter_size)
    return dout

time_start = time.time()
Y_cython, cache_cython = cython_weave.array_weave_fast_forward(X, weave_param)
cython_time = time.time() - time_start
print('cython forward time', cython_time)

time_start = time.time()
Y_python, cache_python = array_weave_forwards(X, weave_param)
python_time = time.time() - time_start
print('python forward time', python_time)
print('Cython Speed Up', python_time/cython_time)

print('Comparrison', (Y_cython[0] == Y_python[0]).mean())

time_start = time.time()
dx_cython = cython_weave.array_weave_fast_backward(out, cache_cython)
cython_time = time.time() - time_start
print('cython forward time', cython_time)

time_start = time.time()
dx_python = array_weave_backwards(out, cache_python)
python_time = time.time()- time_start
print('python forward time', python_time)
print('Cython Speed Up', python_time/cython_time)

print('Comparison', (dx_cython == dx_python).mean())


('cython forward time', 0.030181884765625)
('python forward time', 0.08681797981262207)
('Cython Speed Up', 2.876492985338726)
('Comparrison', 1.0)
('cython forward time', 0.0074939727783203125)
('python forward time', 0.08642911911010742)
('Cython Speed Up', 11.533150928989565)
('Comparison', 1.0)


In [11]:
dx_python = array_weave_backwards(Y_python,cache_python)
dx_cython = cython_array_weave_backwards()

array([[[[-3.74824132e-01, -7.61270742e-02,  5.02675564e-01, ...,
          -2.17780355e+00,  1.42508890e+00,  2.57875763e-01],
         [ 7.95433010e-01,  9.92773565e-01,  2.01517736e-01, ...,
           1.21830032e+00,  3.26626466e-01, -6.99389460e-01],
         [ 1.38472847e+00,  1.36021482e+00,  5.06562598e-01, ...,
          -1.21642263e-01,  5.38431896e-01,  1.05030881e+00],
         ...,
         [-8.45520737e-01, -3.92943984e-01,  5.43114572e-01, ...,
          -1.61278930e-01,  1.05146653e+00,  1.03766107e-01],
         [ 1.10627834e+00,  5.51069818e-02, -8.05603227e-02, ...,
           1.60128064e+00,  6.95825580e-01,  2.30957741e-01],
         [ 1.19046762e+00, -1.69810030e+00,  1.09301231e+00, ...,
           1.16165858e+00,  8.05122932e-01,  4.56834521e-01]],

        [[-3.11390852e+00,  1.10571293e+00,  1.22561195e+00, ...,
          -2.66083160e-01,  1.55689737e+00,  6.68584183e-01],
         [-1.65974568e+00,  4.03107600e-01,  5.93313744e-02, ...,
          -6.79270376e

In [4]:
cython_weave.array_weave_fast_backwards

<function cython_weave.array_weave_fast_backwards>